In [ ]:
# -*- coding: utf-8 -*-
"""
Created on 30/05/2023

@author: Abdelmalek H
"""
import os
import numpy as np
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import Sequence
from sklearn.model_selection import KFold
from keras.applications.vgg19 import VGG19
from keras.layers import Flatten, Dense
from keras.models import Model
from keras.optimizers import Adam


class CustomDataGenerator(Sequence):
    """Custom data generator for image and age data."""

    def __init__(self, image_paths, ages, batch_size, image_size, augment):
        self.image_paths = image_paths
        self.ages = ages
        self.batch_size = batch_size
        self.image_size = image_size
        self.augment = augment
        if self.augment:
            self.datagen = ImageDataGenerator(
                rotation_range=30,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                vertical_flip=True,
                fill_mode='nearest'
            )

    def __len__(self):
        """Get the number of batches per epoch."""
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        """Get a batch of images and ages."""
        batch_image_paths = self.image_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_images = []
        batch_ages = []

        for i, image_path in enumerate(batch_image_paths):
            image, age = self.load_and_preprocess_image(image_path)
            augmented_image = self.apply_image_augmentation(image)
            batch_images.append(augmented_image)
            batch_ages.append( batch_ages.append(1 if Age>18 else 0))

        batch_images = np.array(batch_images)
        batch_ages = np.array(batch_ages)

        return batch_images, batch_ages

    def load_and_preprocess_image(self, image_path):
        """Load and preprocess an image from the given path."""
        npzfile = np.load(image_path)
        image = npzfile['image']
        Age = np.load(image_path)['Age']

        image_resized = cv2.resize(image, self.image_size)
        image_norm = (image_resized.astype('float32') - np.min(image_resized)) / (
                    np.max(image_resized) - np.min(image_resized))

        if len(image.shape) == 2:
            image_tri = np.repeat(image_norm[:, :, np.newaxis], 3, axis=2)
        else:
            image_tri = image_norm

        return image_tri, Age

    def apply_image_augmentation(self, image):
        """Apply image augmentation on the given image."""
        if self.augment:
            augmented_image = self.datagen.random_transform(image)
        else:
            augmented_image = image
        return augmented_image


if __name__ == "__main__":
    # Define paths and parameters
    train_path = r'E:/Papier/AIAGE/Base/reseau de neurones/save/train'
    test_path = r'E:/Papier/AIAGE/Base/reseau de neurones/save/test'
    batch_size = 32
    image_size = (224, 224)
    num_epochs = 10
    k = 5
    seed = 7
    np.random.seed(seed)

    # Load image paths and ages
    train_image_paths = [os.path.join(root, f) for root, dirs, files in os.walk(train_path) for f in files]
    train_ages = [np.load(image_path)['Age'] for image_path in train_image_paths]
    train_ages_binary = [1 if np.load(image_path)['Age'] > 18 else 0 for image_path in train_image_paths]
    test_image_paths = [os.path.join(root, f) for root, dirs, files in os.walk(test_path) for f in files]
    test_ages = [np.load(image_path)['Age'] for image_path in test_image_paths]
    test_ages_binary = [1 if np.load(image_path)['Age'] > 18 else 0 for image_path in test_image_paths]

    # Initialize the KFold object
    kfold = KFold(n_splits=k, shuffle=True, random_state=seed)

    # Initialize variables
    cv_scores = []

    # Loop through the folds in KFold and train the model
    for i, (train_index, test_index) in enumerate(kfold.split(train_image_paths, train_ages)):
        print("======================================")
        print("Iteration = ", i+1)

        # Split the data into train and validation sets
        X_train, X_val = np.array(train_image_paths)[train_index], np.array(train_image_paths)[test_index]
        y_train, y_val = np.array(train_ages)[train_index], np.array(train_ages)[test_index]

        # Create data generators
        train_data_generator = CustomDataGenerator(X_train, y_train, batch_size, image_size, augment=True)
        val_data_generator = CustomDataGenerator(X_val, y_val, batch_size, image_size, augment=False)
        test_data_generator = CustomDataGenerator(test_image_paths, test_ages, batch_size, image_size, augment=False)

#         # Load the VGG19 model
          
#         base_model = VGG19(weights='//chu-lyon.fr/bureautique/TOUS_COMMUNS/BIOSTAT_COMMUN/etude60/6458 stage Perla El Khoueiry/analyse/AI_LA/Perla/reseaux-Age_binaire/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
#                        include_top=False, input_shape=(224, 224, 3))

#         # Set layers in the model as trainable
#         for layer in base_model.layers:
#             layer.trainable = False

#         # Add layers on top of the VGG19 model
#         x = base_model.output
#         x = Flatten()(x)
#         x = Dense(256, activation='relu')(x)
#         x = Dense(128, activation='relu')(x)
#         output = Dense(1, activation='linear')(x)
#         model = Model(inputs=base_model.input, outputs=output)
        pretrained_model = load_model('//chu-lyon.fr/bureautique/TOUS_COMMUNS/BIOSTAT_COMMUN/etude60/6458 stage Perla El Khoueiry/analyse/AI_LA/Perla/model_sauvegarder/VGG19_continue.h5')
        for layer in pretrained_model.layers:
            layer.trainable = False
#     Get the output of the previous layer
        previous_output = pretrained_model.layers[-2].output

   # Add a new dense layer with the desired number of units or classes 
        new_output = Dense(1, activation='sigmoid')(previous_output)

   # Create the new model with the updated last layer 
        model = Model(inputs=pretrained_model.input, outputs=new_output)

        # Compile the model with specified loss function, optimizer and metrics
        model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['precision'])

        # Print the summary of the model
        model.summary()

        # Train the model
        history = model.fit(train_data_generator, epochs=num_epochs, validation_data=val_data_generator)

        # Evaluate the model's performance
        scores = model.evaluate(test_data_generator)

        # Append score to cv_scores list
        cv_scores.append(scores[1] * 100)

        # Print performance metrics and history
        print("cross_entropy: %.2f" % (scores[0]))
        print("precision: %.2f" % (scores[1]))

    # Calculate the mean and standard deviation of the performance metrics
    accuracy = np.mean(cv_scores)
    std = np.std(cv_scores)
    print("Accuracy: %.2f%% (+/- %.2f%%)" % (accuracy, std))